In [15]:
from gputools import deconv

In [16]:
deconv._multiply_inplace

AttributeError: module 'gputools.deconv' has no attribute '_multiply_inplace'

In [1]:
import sys
sys.path.append('/Users/marvin/Documents/projects/z1regfus')
import io_utils
import numpy as np
import dipy_multiview as dm
import tifffile

INFO: successfully registered image array for dask distributed serialization


In [7]:
info = dm.getStackInfoFromCZI('/Users/marvin/data/dbspim/20140911_cxcr7_wt/wt_01.czi')
ws = io_utils.process_input_element('/Users/marvin/data/dbspim/20140911_cxcr7_wt/small/mv_000_000_c00_w.image.h5')
osps = [{'origin':info['origins'][iv][::-1],'spacing':info['spacing'][::-1],'size':info['sizes'][iv][::-1]} for iv in range(4)]
ts = np.array([io_utils.process_input_element('/Users/marvin/data/dbspim/20140911_cxcr7_wt/small/mv_transf_view_000_000_v%03d_c00.imagear.h5' %i) for i in range(4)])
params = io_utils.process_input_element('/Users/marvin/data/dbspim/20140911_cxcr7_wt/mv_params0_000_000.prealignment.h5')
sp = io_utils.process_input_element('/Users/marvin/data/dbspim/20140911_cxcr7_wt/mv_stack_props_000_000.dict.h5')

getting stack info
(4, 1, 1, 1, 1, 2, 1, 204, 1920, 1400, 1)


In [4]:
ts[0].shape

(120, 251, 147)

In [53]:
from gputools import OCLArray, OCLProgram, get_device
from gputools import convolve, fft_convolve, fft, fft_plan
from gputools import OCLElementwiseKernel

_multiply_inplace = OCLElementwiseKernel(
        "float *a, float * b",
        "a[i] = a[i] * b[i]",
    "mult_inplace")

_divide_inplace = OCLElementwiseKernel(
        "float *a, float * b",
        "b[i] = a[i]*b[i]/(b[i]*b[i]+0.001f)",
    "divide_inplace")


_complex_multiply = OCLElementwiseKernel(
        "cfloat_t *a, cfloat_t * b,cfloat_t * res",
        "res[i] = cfloat_mul(a[i],b[i])",
    "mult")

_complex_multiply_inplace = OCLElementwiseKernel(
        "cfloat_t *a, cfloat_t * b",
        "a[i] = cfloat_mul(a[i],b[i])",
    "mult_inplace")

_complex_divide = OCLElementwiseKernel(
        "cfloat_t *a, cfloat_t * b,cfloat_t * res",
        "res[i] = cfloat_divide(b[i],a[i])",
    "div")

_complex_divide_inplace = OCLElementwiseKernel(
        "cfloat_t *a, cfloat_t * b",
        "b[i] = cfloat_divide(a[i],b[i])",
    "divide_inplace")

def cpu(a,b):
    res = np.fft.fft(a)*np.fft.fft(b)
    ires = np.fft.ifft(res)
    return np.abs(ires)

def gpu(a,b):
    a_g = OCLArray.from_array(a.astype(np.float32))
    b_g = OCLArray.from_array(b.astype(np.float32))
    
    _multiply_inplace(a_g,b_g)
    return 

def gpuf(a,b):
    a_g = OCLArray.from_array(a.astype(np.complex64))
    b_g = OCLArray.from_array(b.astype(np.complex64))
    
    fft(a_g, inplace=True)
    fft(b_g, inplace=True)
    
    _complex_multiply_inplace(a_g,b_g)
    fft(a_g, inplace=True, inverse=True)
    res = np.abs(a_g.get())
#     _multiply_inplace(a_g,b_g)
    return res

In [55]:
n = 10
a = np.random.randint(0,100,(n,n))
b = np.random.randint(0,100,(n,n))

In [58]:
# convolution
n = 11
img = np.zeros((n,n)).astype(np.float32)
img[n//2,n//2] = 1.

k = np.zeros((5,5)).astype(np.float32)
k[1:-1,1:-1] = 1.

In [ ]:
def test(a,b):
    

In [68]:
from gputools import convolve
convolve(img,k)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
def cpu_convolve(img,k):
    np

In [56]:
rc = cpu(a,b)
rc

array([[16222., 15613., 16061., 18563., 17927., 16077., 10881., 16193.,
        16156., 19082.],
       [20204., 22277., 20480., 23552., 16579., 20872., 20205., 21617.,
        23849., 18915.],
       [18938., 18308., 12891., 19002., 12777., 18062., 18440., 15321.,
        21459., 16191.],
       [23261., 22918., 26022., 24422., 27334., 24585., 24625., 25914.,
        20187., 26257.],
       [35525., 32077., 34322., 30508., 35329., 34648., 34943., 32913.,
        30397., 33352.],
       [27883., 30559., 28703., 29162., 28822., 27623., 28528., 30589.,
        28270., 29971.],
       [22802., 24291., 25763., 28593., 25549., 24692., 21077., 21134.,
        20775., 24192.],
       [28342., 31887., 33320., 30488., 33138., 30194., 28090., 28902.,
        24240., 30417.],
       [14766., 13741., 18086., 15029., 14102., 17381., 14098., 15789.,
        13690., 17784.],
       [25202., 30234., 27164., 31344., 25294., 32544., 29276., 29950.,
        26484., 31788.]])

In [42]:
fft?

In [57]:
rg = gpuf(a,b)
rg

array([[245764.84, 239456.72, 247676.53, 242508.23, 229876.25, 231844.45,
        239761.95, 242446.97, 247128.2 , 232632.34],
       [240876.3 , 238448.83, 242534.39, 253889.77, 237377.33, 238112.25,
        247533.08, 240110.36, 253751.69, 242389.95],
       [246306.31, 233671.  , 242513.5 , 245750.23, 232150.  , 237736.03,
        224504.45, 236531.1 , 242565.9 , 236097.36],
       [250193.14, 224354.56, 236776.61, 233933.88, 236560.69, 234652.1 ,
        229954.45, 240543.78, 250671.7 , 243212.19],
       [241466.25, 236182.1 , 233975.22, 243133.38, 235924.14, 256263.61,
        231467.67, 250911.8 , 234289.36, 247277.33],
       [250178.9 , 232564.17, 241097.84, 229083.88, 235736.27, 231392.62,
        235085.9 , 235943.56, 225726.5 , 245113.94],
       [249087.14, 227884.36, 246665.1 , 224635.4 , 244102.08, 254008.5 ,
        248169.88, 240026.75, 232124.12, 239954.8 ],
       [235411.88, 245186.14, 234556.48, 239315.6 , 243081.73, 235817.81,
        242450.62, 219841.45, 229484.

In [21]:
import importlib
import deconv_gpu as dg
dm = importlib.reload(dm)
dg = importlib.reload(dg)
# imdm = dm.fuse_LR_with_weights_np(ts,params,sp,2,weights=ws)
imdg = dg.fuse_LR_with_weights_gpu(ts,params,sp,2,weights=ws)

Iteration 0
convergence: 0.0018652081489562988
Iteration 1
convergence: 0.00011944770812988281
Done deconvolving


In [2]:
def fuse_LR_with_weights_np(
        views,
        params,
        stack_properties,
        num_iterations = 25,
        sz = 4,
        sxy = 0.5,
        tol = 5e-5,
        weights = None,
        # orig_prop_list = None,
):
    """
    Combine
    - LR multiview fusion
      (adapted from python code given in https://code.google.com/archive/p/iterative-fusion/
       from publication https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3986040/)
    - DCT weights

    This addresses the problems that
    1) multi-view deconvolution is highly dependent on high precision registration
    between views. However, an affine transformation is often not enough due to
    optical aberrations and results in poor overlap.
    2) due to scattering, the psf strongly varies within each view

    In the case of highly scattering samples, FFT+elastix typically results in good
    registration accuracy in regions of good image quality (those with small psfs
    and short optical paths through the sample). These regions are found using a DCT
    quality measure and weighted accordingly. Therefore, to reduce the contribution
    to multi-view LR of unwanted regions in the individual views, the weights are
    applied in each iteration during convolution with the psf.

    Adaptations and details:
    - convolve views in original space
     - recapitulates imaging process and trivially deals with view parameters
     - allows for iterative raw data reconstruction without deconvolution
     - disadvantage: slow in current implementation
    - apply DCT weights in each blurring iteration to account for strong scattering
    - simulate convolution by psf with gaussian blurring
    - TV regularisation not working yet, to be optimised (Multiview Deblurring for 3-D Images from
Light-Sheet-Based Fluorescence Microscopy, https://ieeexplore.ieee.org/document/6112225)

    Interesting case: sz,sxy=0
    - formally no deconvolution but iterative multi-view raw data reconstruction

    works well:
    - sz6 it 10, some rings
    - sz5 it 20, looks good (good compromise between sz and its)
    - sz4 it 30, good and no rings

    :param views: original views
    :param params: parameters mapping views into target space
    :param stack_properties: properties of target space
    :param num_iterations: max number of deconvolution iterations
    :param sz: sigma z
    :param sxy: sigma xy
    :param tol: convergence threshold
    :return:
    """

    psfs =  np.array([get_psf(params[ip], stack_properties, sz, sxy) for ip in range(len(params))])

    noisy_multiview_data = np.array(views)

    """
    Time for deconvolution!!!
    """

    estimate = np.sum([weights[i]*views[i] for i in range(len(params))],0).astype(np.float32)

    curr_imsum = np.sum(estimate)

    masks = np.array([weights[ip]>1e-5 for ip in range(len(params))])

    # # erode weights to produce final weights
    # pixels = int(sz*4/stack_properties['spacing'][0])
    # weights = np.array([ndimage.grey_erosion(w,size=pixels) for w in weights])

    # masks = np.array([ndimage.binary_erosion(mask,iterations=2) for mask in masks])

    from numpy import fft
    shape = estimate.shape

    psfs_ft = []
    for ip in range(len(psfs)):
        kernel_pad = np.pad(psfs[ip], [[0, shape[i]+1] for i in range(3)], mode='constant')
        psfs_ft.append(fft.rfftn(kernel_pad))

    kshape = psfs[0].shape

    def blur_with_ftpsfind(img,iview):

        # manual convolution is faster and better than signal.fftconvolve
        # - kernels can be computed before
        # - image padding can be done using reflection
        # - probably also: no additional checking involved
        # https://dsp.stackexchange.com/questions/43953/looking-for-fastest-2d-convolution-in-python-on-a-cpu
        # https://github.com/scipy/scipy/pull/10518

        img_ft = np.pad(img, [[0, kshape[i]+1] for i in range(3)], mode='reflect')
        out_shape = img_ft.shape
        img_ft = fft.rfftn(img_ft)
        img_ft = psfs_ft[iview] * img_ft
        img_ft = fft.irfftn(img_ft,s=out_shape)
        # img_ft = img_ft[kshape[0] // 2:-kshape[0] // 2, kshape[1] // 2:-kshape[1] // 2, kshape[2] // 2:-kshape[2] // 2]
        img_ft = img_ft[kshape[0] // 2:-kshape[0] // 2 - 1, kshape[1] // 2:-kshape[1] // 2 - 1, kshape[2] // 2:-kshape[2] // 2 - 1]

        img_ft[img_ft<0] = 0

        return img_ft

    expected_data = np.zeros(noisy_multiview_data.shape,dtype=np.float32)

    i = 0
    while 1:
        print("Iteration", i)

        """
        Construct the expected data from the estimate
        """

        # expected_data = []
        for ip, p in enumerate(psfs):
            expected_data[ip] = blur_with_ftpsfind(estimate, ip)


        "Done constructing."
        """
        Take the ratio between the measured data and the expected data.
        Store this ratio in 'expected_data'
        """
        expected_data = noisy_multiview_data / (expected_data + 1e-6)

        # multiply with mask to reduce border artifacts

        expected_data *= masks

        # for ip in range(len(params)):
        #     expected_data[ip] = expected_data[ip] * sitk.Cast(weights[ip]>0,sitk.sitkFloat32)
        """
        Apply the transpose of the expected data operation to the correction factor
        """
        correction_factor = expected_data[0] * 0.
        for ip, p in enumerate(psfs):

            # o = blur_func(multiview_data[ip],p,orig_prop_list[ip],stack_properties,sz,sxy)
            o = blur_with_ftpsfind(expected_data[ip], ip)

            if weights is not None:
                o = o * weights[ip]

            correction_factor += o

        """
        Multiply the old estimate by the correction factor to get the new estimate
        """

        estimate = estimate * correction_factor

        estimate = np.clip(estimate,0,2**16-1)

        # if num_iterations < 1:
        new_imsum = np.sum(estimate)
        conv = np.abs(1-new_imsum/curr_imsum)
        print('convergence: %s' %conv)

        if conv < tol and i>=10: break
        if i >= num_iterations-1: break

        curr_imsum = new_imsum
        i += 1

        """
        Update the history
        """
    print("Done deconvolving")

    estimate = ImageArray(estimate.astype(np.uint16),spacing=stack_properties['spacing'],origin=stack_properties['origin'])

    return estimate